# 1: Set Up Your Environment

## 1.Install Docker: Make sure Docker is installed and running on your machine.

## 2.Set Up Docker Compose for PySpark:
    In the same folder as your CSV file, create a docker-compose.yml file
## 3.Start Docker
Run the command `docker-compose up` to start the Docker container.

# 2:  PySpark Code


In [1]:
#Import Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, to_date, month, desc
from pyspark.sql import functions as F
from pyspark.sql.window import Window


In [2]:
#Initialize Spark Session
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


In [3]:
#Load the CSV File
file_path = "sales_data.csv"  
df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)


In [4]:
#Preview the Data
df.show(5)
df.printSchema()
print(f"Total Rows: {df.count()}")


+--------------+---------+----------+----------+------+-----------+
|transaction_id|client_id|product_id|      date|amount|   category|
+--------------+---------+----------+----------+------+-----------+
|             1|      101|      P001|2023-01-15|  NULL|Electronics|
|             2|      102|      P002|2023-02-20|    80|   Clothing|
|             3|      103|      P003|2023-03-15|   230|Electronics|
|             4|      101|      P001|2023-04-10|    75|Electronics|
|             5|      104|      P002|2023-05-25|   180|   Clothing|
+--------------+---------+----------+----------+------+-----------+
only showing top 5 rows

root
 |-- transaction_id: integer (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- amount: integer (nullable = true)
 |-- category: string (nullable = true)

Total Rows: 17


# 3: Data Cleaning and Filtering

In [5]:
#Filter Transactions with Amount > 100
filtered_df = df.filter(col("amount") > 100)


In [6]:
#Replace Null Values in amount and category Columns

cleaned_df = df.fillna({"amount": 0, "category": "Unknown"})


In [7]:
#Convert date Column to Date Format
date_df = cleaned_df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))


# 4: Perform Analysis

In [8]:
#Calculate Total Sales
total_sales = date_df.agg(sum("amount")).first()[0]
print(f"Total Sales: {total_sales}")


Total Sales: 2080


In [9]:
#Total Sales by Category
sales_by_category = date_df.groupBy("category").agg(sum("amount").alias("total_sales"))
sales_by_category.show()


+-----------+-----------+
|   category|total_sales|
+-----------+-----------+
|Electronics|       1190|
|   Clothing|        450|
|    Unknown|        390|
|Accessories|         50|
+-----------+-----------+



In [10]:
#Sales by Month
sales_by_month = date_df.withColumn("month", month(col("date"))) \
    .groupBy("month").agg(sum("amount").alias("monthly_sales"))
sales_by_month.show()


+-----+-------------+
|month|monthly_sales|
+-----+-------------+
|   12|          220|
|    1|          140|
|    6|           50|
|    3|          480|
|    5|          370|
|    9|          200|
|    4|          160|
|    8|           90|
|    7|          120|
|   10|           70|
|   11|            0|
|    2|          180|
+-----+-------------+



# 5: Identify Top-Selling Products

In [11]:
#Top 5 Products by Sales Amount
top_products = date_df.groupBy("product_id").agg(sum("amount").alias("total_sales")) \
    .orderBy(desc("total_sales")).limit(5)
top_products.show()


+----------+-----------+
|product_id|total_sales|
+----------+-----------+
|      P003|        900|
|      P002|        640|
|      P001|        490|
|      P004|         50|
+----------+-----------+



In [12]:
#Top Product per Category
window_spec = Window.partitionBy("category").orderBy(desc("total_sales"))
top_product_by_category = date_df.groupBy("category", "product_id") \
    .agg(sum("amount").alias("total_sales")) \
    .withColumn("rank", F.rank().over(window_spec)) \
    .filter(col("rank") == 1)
top_product_by_category.show()


+-----------+----------+-----------+----+
|   category|product_id|total_sales|rank|
+-----------+----------+-----------+----+
|Accessories|      P004|         50|   1|
|   Clothing|      P002|        450|   1|
|Electronics|      P003|        700|   1|
|    Unknown|      P003|        200|   1|
+-----------+----------+-----------+----+

